In [1]:
import numpy as np
from random import shuffle
# import scipy.ndimage
import scipy.io as io
# import argparse

In [2]:
input_mat = io.loadmat('Indian_pines.mat')['indian_pines']
target_mat = io.loadmat('Indian_pines_gt.mat')['indian_pines_gt']

In [3]:
PATCH_SIZE = 3
HEIGHT = input_mat.shape[0]
WIDTH = input_mat.shape[1]
BAND = input_mat.shape[2]
CLASSES = [] 
COUNT = 200
OUTPUT_CLASSES = np.max(target_mat)
print (OUTPUT_CLASSES)

16


In [4]:
input_mat = input_mat.astype(float)
input_mat -= np.min(input_mat)
input_mat /= np.max(input_mat)
list_labels = [2,3,5,6,8,10,11,12,14]
train_idx = [178, 178, 178, 177, 177, 178, 178, 178, 178]

In [5]:
def Patch(height_index,width_index):
    """
    Returns a mean-normalized patch, the top left corner of which 
    is at (height_index, width_index)
    
    Inputs: 
    height_index - row index of the top left corner of the image patch
    width_index - column index of the top left corner of the image patch
    
    Outputs:
    mean_normalized_patch - mean normalized patch of size (PATCH_SIZE, PATCH_SIZE) 
    whose top left corner is at (height_index, width_index)
    """
    transpose_array = input_mat
    height_slice = slice(height_index, height_index+PATCH_SIZE)
    width_slice = slice(width_index, width_index+PATCH_SIZE)
    patch = transpose_array[:, height_slice, width_slice]
    mean_normalized_patch = []
    for i in range(patch.shape[0]):
        mean_normalized_patch.append(patch[i] - MEAN_ARRAY[i]) 
    
    return np.array(mean_normalized_patch)

In [6]:
MEAN_ARRAY = np.ndarray(shape=(BAND,),dtype=float)
new_input_mat = []
input_mat = np.transpose(input_mat,(2,0,1))
print(input_mat.shape)
for i in range(BAND):
    MEAN_ARRAY[i] = np.mean(input_mat[i,:,:])
    new_input_mat.append(np.pad(input_mat[i,:,:],int(PATCH_SIZE/2),'constant',constant_values = 0))
    
print (np.array(new_input_mat).shape)

(220, 145, 145)
(220, 147, 147)


In [7]:
input_mat = np.array(new_input_mat)


for i in range(OUTPUT_CLASSES):
    CLASSES.append([])
count = 0
image = []
image_label = []
for i in range(HEIGHT):
    for j in range(WIDTH):
        curr_inp = Patch(i,j)
        curr_tar = target_mat[i , j]
        if(curr_tar!=0): #Ignore patches with unknown landcover type for the central pixel
            CLASSES[curr_tar-1].append(curr_inp)
            count += 1
print (count)

10249


In [8]:
TRAIN_PATCH,TRAIN_LABELS,TEST_PATCH,TEST_LABELS,VAL_PATCH, VAL_LABELS = [],[],[],[],[],[]
FULL_TRAIN_PATCH = []
FULL_TRAIN_LABELS = []
count = 0
for i, data in enumerate(CLASSES):
    if i+1 in list_labels:
        shuffle(data)
        TRAIN_PATCH += data[:train_idx[count]]
        TRAIN_LABELS += [count]*train_idx[count]
        VAL_PATCH += data[train_idx[count]:200]
        VAL_LABELS += [count]*(200-train_idx[count])
        TEST_PATCH += data[200:]
        TEST_LABELS += [count]*(len(data) - 200)
        count += 1

FULL_TRAIN_LABELS = TRAIN_LABELS + VAL_LABELS
FULL_TRAIN_PATCH = TRAIN_PATCH + VAL_PATCH

TRAIN_LABELS = np.array(TRAIN_LABELS)
TRAIN_PATCH = np.array(TRAIN_PATCH)
TEST_PATCH = np.array(TEST_PATCH)
TEST_LABELS = np.array(TEST_LABELS)
VAL_PATCH = np.array(VAL_PATCH)
VAL_LABELS = np.array(VAL_LABELS)
FULL_TRAIN_LABELS = np.array(FULL_TRAIN_LABELS)
FULL_TRAIN_PATCH = np.array(FULL_TRAIN_PATCH)

train_idx = list(range(len(TRAIN_PATCH)))
shuffle(train_idx)
TRAIN_PATCH = TRAIN_PATCH[train_idx]
TRAIN_LABELS = TRAIN_LABELS[train_idx]
test_idx = range(len(TEST_PATCH))
TEST_PATCH = TEST_PATCH[test_idx]
TEST_LABELS = TEST_LABELS[test_idx]
val_idx = list(range(len(VAL_PATCH)))
shuffle(val_idx)
VAL_PATCH = VAL_PATCH[val_idx]
VAL_LABELS = VAL_LABELS[val_idx]
full_train_idx = shuffle(list(range(len(FULL_TRAIN_PATCH))))
FULL_TRAIN_PATCH = FULL_TRAIN_PATCH[full_train_idx]
FULL_TRAIN_LABELS = FULL_TRAIN_LABELS[full_train_idx]

In [9]:
train = {}
train["train_patch"] = TRAIN_PATCH
train["train_labels"] = TRAIN_LABELS
io.savemat("indian_pines_Train_patch_" + str(PATCH_SIZE) + ".mat", train)
print (TRAIN_PATCH.shape)


test = {}
test["test_patch"] = TEST_PATCH
test["test_labels"] = TEST_LABELS
io.savemat("indian_pines_Test_patch_" + str(PATCH_SIZE) + ".mat", test)
print (TEST_PATCH.shape)

val = {}
val["val_patch"] = VAL_PATCH
val["val_labels"] = VAL_LABELS
io.savemat("indian_pines_Val_patch_" + str(PATCH_SIZE) + ".mat", val)
print (VAL_PATCH.shape)

full_train = {}
full_train["train_patch"] = FULL_TRAIN_PATCH
full_train["train_labels"] = FULL_TRAIN_LABELS
io.savemat("indian_pines_Full_Train_patch_" + str(PATCH_SIZE) + ".mat", full_train)
print (FULL_TRAIN_LABELS.shape)

(1600, 220, 3, 3)
(7434, 220, 3, 3)
(200, 220, 3, 3)
(1, 1800)


In [10]:
# require 'nn'
# require 'optim'
# matio = require 'matio'
# require 'xlua'
# require 'pl'
# require 'paths'
# require 'torch'
# require 'math'

# local opt = lapp[[
#    --development      (default 1)                    Use development dataset/ Whole training dataset 
#    -s,--save          (default "logs/")               subdirectory to save logs
#    -p,--plot                                         plot while training
#    -o,--optimization  (default "Adam")               optimization: SGD | LBFGS | Adam
#    -l,--learningRate  (default 0.0005)              learning rate, for SGD only
#    -b,--batchSize     (default 200)                  batch size
#    -m,--momentum      (default 0)                    momentum, for SGD only
#    -i,--maxIter       (default 8000)                 maximum nb of iterations per batch, for LBFGS
#    --coefL1           (default 0)                    L1 penalty on the weights
#    --coefL2           (default 0)                    L2 penalty on the weights
#    -t,--type          (default "cpu")                GPU or CPU
#    --network          (default "MLP")                MLP or CNN
#    --patch_size       (default 3)                    patch size of tthe image
#    --nbands           (default 10)                   number of bands
#    --block1_conv1     (default 3333)                 number of filters in the 1*1 convolution (here, 3333 is the sentinel value)
# ]]
development = 1
save = 'logs/'
learningRate = 0.0005
batchSize = 200
momentum = 0
maxIter = 8000
coefL1 = 0
coefL2 = 0
patch_size = 3
nbands = 10
block1_conv1 = 3333

In [11]:
nval = 200
nclasses = 9

In [12]:
test_datamat = io.loadmat('indian_pines_Test_patch_3.mat')
test_data = test_datamat['test_patch']
test_labels = test_datamat['test_labels'].transpose()
channels = test_data.shape[2] #test_data.shape[1]

In [13]:
train_datamat = io.loadmat('indian_pines_Train_patch_3.mat')
train_data = train_datamat['train_patch']
train_labels = train_datamat['train_labels'].transpose()

val_data = io.loadmat("indian_pines_Val_patch_3.mat")['val_patch']
val_labels = io.loadmat("indian_pines_Val_patch_3.mat")['val_labels'].transpose()

In [14]:
if block1_conv1 == 3333:
    block1_conv1 = channels

while (block1_conv1 % nbands == 0):
    nbands = nbands + 1
    print("Number of parallel networks reinitialized to ",nbands)

In [23]:
train_data.shape

(1600, 220, 3, 3)

In [25]:
train_labels.shape

(1600, 1)

In [28]:
trainset = {}
trainset["data"] = train_data
trainset["labels"] = train_labels

testset = {}
testset["data"] = test_data
testset["labels"] = test_labels

valset = {}
valset["data"] = val_data
valset["labels"] = val_labels

# print(trainset)
# print(testset)
# print(valset)

In [29]:
band_size = block1_conv1/nbands
band_size

0.3

In [33]:
from keras.models import Sequential
from keras.layers import Dense, Reshape, ReLU, Conv1D, Conv2D

model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.add(Conv2D(64, kernel_size=3, activation=’relu’, input_shape=(28,28,1)))
model.add(Conv2D(32, kernel_size=3, activation=’relu’))
model.add(Flatten())
model.add(Dense(10, activation=’softmax’))


model = nn.Sequential()

model:add(nn.Reshape(opt.band_size, opt.patch_size * opt.patch_size))
model:add(nn.TemporalConvolution(opt.patch_size * opt.patch_size, 20, 3, 1))
model:add(nn.ReLU())
model:add(nn.TemporalConvolution(20, 20, 3, 1))
model:add(nn.ReLU())
model:add(nn.TemporalConvolution(20, 10, 3, 1))
model:add(nn.ReLU())
model:add(nn.TemporalConvolution(10, 5, 5, 1))
model:add(nn.ReLU())
model:add(nn.Reshape((opt.band_size-10) * 5, 1))
parallel_model = nn.Parallel(2, 2)
for i = 1, opt.nbands do
    parallel_model:add(model:clone())
end


In [36]:
model = Sequential()

model.add(Reshape((band_size, patch_size*patch_size)))
model.add(Conv1D( 20, kernel_size=3, strides=1))
model.add(ReLU())
model.add(Conv1D( 20, kernel_size=3, strides=1))
model.add(ReLU())
model.add(Conv1D( 10, kernel_size=3, strides=1))
model.add(ReLU())
model.add(Conv1D( 5, kernel_size=3, strides=1))
model.add(ReLU())
model.add(Reshape(((band_size-10)*5, 1)))


net = nn.Sequential()
net:add(nn.SpatialConvolution(opt.channels, opt.block1_conv1, 1, 1))
net:add(nn.ReLU())
net:add(nn.Reshape(opt.nbands, opt.band_size, opt.patch_size*opt.patch_size))
net:add(parallel_model)
net:add(nn.Reshape(opt.nbands*(opt.band_size-10)*5))
net:add(nn.Linear(opt.nbands*(opt.band_size-10)*5, 100))
net:add(nn.ReLU())
net:add(nn.Dropout())
net:add(nn.Linear(100, opt.nclasses))
net:add(nn.LogSoftMax())
__SpatialConvolution : init(nInputPlane, nOutputPlane, kW, kH, dW, dH, padW, padH)


In [43]:
net = Sequential()
net.add(Conv2D( block1_conv1, kernel_size = 1, strides = 1))
net.add(ReLU())
net.add(Reshape(( nbands, band_size, patch_size**2)))
# net.add(parallel_model)
# net.add(Reshape(( nbands*(band_size-10)*5))
net.add(Dense(100))
net.add(ReLU())
net.add(Dense(9, activation="softmax"))

In [ ]:
# setmetatable(trainset, {__index = function(self, index)
#              local input = self.data[index]
#              local class = self.labels[index]
#              local labelvector = torch.zeros(opt.nclasses)
#              local label = labelvector
#              label[class[1]+1] = 1
#              local example = {input, label}
#                                    return example
# end})

# setmetatable(testset, {__index = function(self, index)
#              local input = self.data[index]
#              local class = self.labels[index]
#              local labelvector = torch.zeros(opt.nclasses)
#              local label = labelvector
#              label[class[1]+1] = 1
#              local example = {input, label}
#                                    return example
# end})

# setmetatable(valset, {__index = function(self, index)
#              local input = self.data[index]
#              local class = self.labels[index]
#              local labelvector = torch.zeros(opt.nclasses)
#              local label = labelvector
#              label[class[1]+1] = 1
#              local example = {input, label}
#                                    return example
# end})


# function trainset:size()
#     return trainset.data:size(1)
# end
# function testset:size()
#     return testset.data:size(1)
# end
# function valset:size()
#     return valset.data:size(1)
# end


# model = nn.Sequential()

# model:add(nn.Reshape(opt.band_size, opt.patch_size*opt.patch_size))
# model:add(nn.TemporalConvolution(opt.patch_size*opt.patch_size, 20, 3, 1))
# model:add(nn.ReLU())
# model:add(nn.TemporalConvolution(20, 20, 3, 1))
# model:add(nn.ReLU())
# model:add(nn.TemporalConvolution(20, 10, 3, 1))
# model:add(nn.ReLU())
# model:add(nn.TemporalConvolution(10, 5, 5, 1))
# model:add(nn.ReLU())
# model:add(nn.Reshape((opt.band_size-10)*5, 1))

parallel_model = nn.Parallel(2, 2)
for i = 1, opt.nbands do
    parallel_model:add(model:clone())
end

net = nn.Sequential()
net:add(nn.SpatialConvolution(opt.channels, opt.block1_conv1, 1, 1))
net:add(nn.ReLU())
net:add(nn.Reshape(opt.nbands, opt.band_size, opt.patch_size*opt.patch_size))
net:add(parallel_model)
net:add(nn.Reshape(opt.nbands*(opt.band_size-10)*5))
net:add(nn.Linear(opt.nbands*(opt.band_size-10)*5, 100))
net:add(nn.ReLU())
net:add(nn.Dropout())
net:add(nn.Linear(100, opt.nclasses))
net:add(nn.LogSoftMax())

-- Parameter Sharing

parallel_model = net:get(4)
for band = 2, opt.nbands do
  local current_module = parallel_model:get(band)
  current_module:share(parallel_model:get(1), 'weight', 'bias',
                       'gradWeight', 'gradBias')
end

net:training()

criterion = nn.ClassNLLCriterion()

if opt.data == "Salinas" then
  classes = {'0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15'}
else
  classes = {'0', '1', '2', '3', '4', '5', '6', '7', '8'}
end

parameters, gradParameters = net:getParameters()
confusion = optim.ConfusionMatrix(classes)

trainLogger = optim.Logger(paths.concat("./" .. opt.save .. opt.data, 'train.log'))
valLogger = optim.Logger(paths.concat("./" .. opt.save .. opt.data, 'val.log'))
testLogger = optim.Logger(paths.concat("./" .. opt.save .. opt.data, 'test.log'))


function train(dataset)
   -- epoch tracker
   epoch = epoch or 1

   -- local vars
   local time = sys.clock()

   -- do one epoch
   print('<trainer> on training set:')
   print("<trainer> online epoch # " .. epoch .. ' [batchSize = ' .. opt.batchSize .. ']')
   for t = 1,dataset:size(),opt.batchSize do

      local inputs = torch.Tensor(opt.batchSize, opt.channels, opt.patch_size, opt.patch_size)

      local targets = torch.Tensor(opt.batchSize)
      if opt.type == "cuda" then
      	inputs:cuda()
      	targets:cuda()
      end
      local k = 1
      for i = t,math.min(t+opt.batchSize-1,dataset:size()) do
         -- load new sample
         local sample = dataset[i]
         local input = sample[1]:clone()
         local _,target = sample[2]:clone():max(1)
         target = target:squeeze()
         inputs[k] = input
         targets[k] = target
         k = k + 1
      end

      -- create closure to evaluate f(X) and df/dX
      local feval = function(x)
         -- just in case:
         collectgarbage()

         -- get new parameters
         if x ~= parameters then
            parameters:copy(x)
         end

         -- reset gradients
         gradParameters:zero()

         -- evaluate function for complete mini batch
         local outputs = net:forward(inputs)
         local f = criterion:forward(outputs, targets)

         -- estimate df/dW
         local df_do = criterion:backward(outputs, targets)
         net:backward(inputs, df_do)

         -- penalties (L1 and L2):
         if opt.coefL1 ~= 0 or opt.coefL2 ~= 0 then
            -- locals:
            local norm,sign= torch.norm,torch.sign

            -- Loss:
            f = f + opt.coefL1 * norm(parameters,1)
            f = f + opt.coefL2 * norm(parameters,2)^2/2

            -- Gradients:
            gradParameters:add( sign(parameters):mul(opt.coefL1) + parameters:clone():mul(opt.coefL2) )
         end

         -- update confusion
         -- print(inputs:size(1))
         for i = 1,opt.batchSize do
            confusion:add(outputs[i], targets[i])
         end

         -- return f and df/dX
         return f,gradParameters
      end

      -- optimize on current mini-batch
      if opt.optimization == 'LBFGS' then

         -- Perform LBFGS step:
         lbfgsState = lbfgsState or {
            maxIter = opt.maxIter,
            lineSearch = optim.lswolfe
         }
         optim.lbfgs(feval, parameters, lbfgsState)
       
         -- disp report:
         print('LBFGS step')
         print(' - progress in batch: ' .. t .. '/' .. dataset:size())
         print(' - nb of iterations: ' .. lbfgsState.nIter)
         print(' - nb of function evalutions: ' .. lbfgsState.funcEval)

      elseif opt.optimization == 'SGD' then

         -- Perform SGD step:
         sgdState = sgdState or {
            learningRate = opt.learningRate,
            momentum = opt.momentum,
            learningRateDecay = 5e-7
         }
         optim.sgd(feval, parameters, sgdState)
      elseif opt.optimization == "Adam" then

      	 adamState = adamState or {
      	 	learningRate = opt.learningRate,
      	 	momentum = opt.momentum,
      	 	learningRateDecay = 5e-9
      	 }
      	 optim.adam(feval, parameters, adamState)
         -- disp progress
         xlua.progress(t, dataset:size())

      else
         error('unknown optimization method')
      end
   end
   
   -- time taken
   time = sys.clock() - time
   time = time / dataset:size()

   -- print confusion matrix
   print(confusion)
   print('% mean class accuracy (train set)' .. tostring(confusion.totalValid*100))
   trainLogger:add{['% mean class accuracy (train set)'] = confusion.totalValid * 100}
   confusion:zero()
   epoch = epoch + 1
   return (1 - confusion.totalValid)*100
end
best_val = 0
function val(dataset)

    net:evaluate()
    for t = 1, dataset:size(), opt.batchSize do
        xlua.progress(t, dataset:size())
        local inputs = torch.Tensor(opt.batchSize, opt.channels, opt.patch_size, opt.patch_size)
        local targets = torch.Tensor(opt.batchSize)
        local k = 1
        for i = t, math.min(t+opt.batchSize-1, dataset:size()) do
            local sample = dataset[i]
            local input = sample[1]:clone()
            local _,target = sample[2]:clone():max(1)
            target = target:squeeze()
            inputs[k] = input
            targets[k] = target
            k = k+1
        end
        
        local preds = net:forward(inputs) -- Computing Loss
        for l = 1, k - 1 do 
            confusion:add(preds[l], targets[l])
        end
    end
    print(confusion)
    if confusion.totalValid > best_val then
        best_val = confusion.totalValid
    end
    print("Best validation accuracy yet :" .. tostring(best_val*100) .. "%")
    valLogger:add{['% mean class accuracy (val set)'] = confusion.totalValid * 100}
    confusion:zero()
    net:training()

end     

best_test = 0
function test(dataset)

    net:evaluate()
    for t = 1, dataset:size(), opt.batchSize do
        xlua.progress(t, dataset:size())
        local inputs = torch.Tensor(opt.batchSize, opt.channels, opt.patch_size, opt.patch_size)
        local targets = torch.Tensor(opt.batchSize)
        local k = 1
        for i = t, math.min(t+opt.batchSize-1, dataset:size()) do
            local sample = dataset[i]
            local input = sample[1]:clone()
            local _,target = sample[2]:clone():max(1)
            target = target:squeeze()
            inputs[k] = input
            targets[k] = target
            k = k+1
        end
        
        local preds = net:forward(inputs) -- Computing Loss
        for l = 1, k - 1 do 
            confusion:add(preds[l], targets[l])
        end
    end
    print(confusion)
    if confusion.totalValid > best_test then
        best_test = confusion.totalValid
        torch.save("./pretrained/final_Block_2_no_fc.t7", net)
    end
    print("Best test accuracy yet :" .. tostring(best_test*100) .. "%")
    testLogger:add{['% mean class accuracy (test set)'] = confusion.totalValid * 100}
    confusion:zero()
    net:training()

end     


for i = 1, opt.maxIter do
    train(trainset)
    if opt.development == 1 then
      val(valset)
    else
      test(testset)
  end
end
